# WPDx Comparison

In [1]:
# importing relevant packages
%run /Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_01_packages.py

In [2]:
# import useful functions
%run /Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_02_functions.py

In [3]:
# fetching hidden api key from desktop stored under 'water_api_key'
%run /Users/thomasadler/Desktop/futuristic-platipus/keys.py

In [4]:
# defining working directory
filepath = '/Users/thomasadler/Desktop/futuristic-platipus/data/'

In [5]:
# setting API endpoint, dataset, key and client profile
socrata_domain = 'data.waterpointdata.org'
socrata_dataset_identifier = '9pn9-g5u4'
socrata_token = os.environ.get(water_api_key)
client = Socrata(socrata_domain, socrata_token, timeout=10)

In [15]:
# creating query to fetch Uganda water points, all columns included
water_uganda_query = """
select
    *
limit
    200000
"""

In [17]:
# query dataset and store in dataframe
results = client.get(socrata_dataset_identifier, query=water_uganda_query)
wpdx_prediction_df_raw = pd.DataFrame.from_records(results)

In [18]:
# keep raw dataset untouched
wpdx_prediction_df = wpdx_prediction_df_raw.copy()

In [19]:
# short look at data
wpdx_prediction_df.head()

,row_id,country_name,status_id,management,install_year,age,source,adm1,adm2,wpdx_id,...,location,count,notes,prediction,prediction_level,geocoded_column,:@computed_region_7zzf_xi3x,water_source,pay,water_tech
0,212904,Uganda,True,Community Management,2000,9,"Ministry of Water and Environment, Uganda",APAC,KWANIA,wpdx-00212904,...,"(1.85898, 32.5605)",1,Village=AJAR,0.775195278,Low Risk,"{'type': 'Point', 'coordinates': [32.5605, 1.8...",164,NaN,NaN,NaN
1,223181,Uganda,True,Community Management,NaN,NaN,"Ministry of Water and Environment, Uganda",KOTIDO,JIE,wpdx-00223181,...,"(2.99382, 34.1998)",1,Village=LORIU,0.744118007,Low Risk,"{'type': 'Point', 'coordinates': [34.1998, 2.9...",164,Dam,NaN,NaN
2,203413,Uganda,True,Community Management,2009,0,"Ministry of Water and Environment, Uganda",ARUA,AYIVU,wpdx-00203413,...,"(3.0963, 30.9006)",1,Village=OMI,0.531849859,Medium Risk,"{'type': 'Point', 'coordinates': [30.9006, 3.0...",164,Shallow well,NaN,NaN
3,203353,Uganda,True,Community Management,1987,22,"Ministry of Water and Environment, Uganda",ARUA,AYIVU,wpdx-00203353,...,"(3.0128, 30.8331)",1,Village=ANDRU,0.641468187,Medium Risk,"{'type': 'Point', 'coordinates': [30.8331, 3.0...",164,Protected spring,NaN,NaN
4,203801,Uganda,True,Community Management,2006,3,"Ministry of Water and Environment, Uganda",ARUA,AYIVU,wpdx-00203801,...,"(2.986, 30.9401)",1,Village=Ogayi,0.839493928,Low Risk,"{'type': 'Point', 'coordinates': [30.9401, 2.9...",164,Protected spring,NaN,NaN


In [20]:
# looking at column datatypes
wpdx_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97268 entries, 0 to 97267
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   row_id                       97268 non-null  object
 1   country_name                 97268 non-null  object
 2   status_id                    97268 non-null  bool  
 3   management                   97268 non-null  object
 4   install_year                 92790 non-null  object
 5   age                          92790 non-null  object
 6   source                       97268 non-null  object
 7   adm1                         97268 non-null  object
 8   adm2                         97268 non-null  object
 9   wpdx_id                      97268 non-null  object
 10  report_date                  97268 non-null  object
 11  country_id                   97268 non-null  object
 12  activity_id                  97268 non-null  object
 13  data_lnk                     97

In [22]:
#choosing relevant columns
wpdx_prediction_df=wpdx_prediction_df[['row_id', 'wpdx_id', 'status_id', 'adm1', 'adm2', 'lat_deg', 'lon_deg', 'location', 'prediction', 'prediction_level' ]]

#check
wpdx_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97268 entries, 0 to 97267
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   row_id            97268 non-null  object
 1   wpdx_id           97268 non-null  object
 2   status_id         97268 non-null  bool  
 3   adm1              97268 non-null  object
 4   adm2              97268 non-null  object
 5   lat_deg           97268 non-null  object
 6   lon_deg           97268 non-null  object
 7   location          97268 non-null  object
 8   prediction        97268 non-null  object
 9   prediction_level  97268 non-null  object
dtypes: bool(1), object(9)
memory usage: 6.8+ MB


In [23]:
#checking missing values
wpdx_prediction_df.isna().sum().sum()

0

In [24]:
#checking duplicates
print(f"Dataset has {wpdx_prediction_df.duplicated().sum()} duplicated rows and {wpdx_prediction_df.T.duplicated().sum()} duplicated columns")

Dataset has 0 duplicated rows and 0 duplicated columns


In [25]:
# convert to number
for col in ['lat_deg', 'lon_deg', 'prediction']:
    float_converter(wpdx_prediction_df, col)

# check
wpdx_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97268 entries, 0 to 97267
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_id            97268 non-null  object 
 1   wpdx_id           97268 non-null  object 
 2   status_id         97268 non-null  bool   
 3   adm1              97268 non-null  object 
 4   adm2              97268 non-null  object 
 5   lat_deg           97268 non-null  float32
 6   lon_deg           97268 non-null  float32
 7   location          97268 non-null  object 
 8   prediction        97268 non-null  float32
 9   prediction_level  97268 non-null  object 
dtypes: bool(1), float32(3), object(6)
memory usage: 5.7+ MB


In [26]:
# convert to binary
wpdx_prediction_df['status_id'] = wpdx_prediction_df['status_id'].map({
    True:
    1,
    False:
    0
})

# check
wpdx_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97268 entries, 0 to 97267
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_id            97268 non-null  object 
 1   wpdx_id           97268 non-null  object 
 2   status_id         97268 non-null  int64  
 3   adm1              97268 non-null  object 
 4   adm2              97268 non-null  object 
 5   lat_deg           97268 non-null  float32
 6   lon_deg           97268 non-null  float32
 7   location          97268 non-null  object 
 8   prediction        97268 non-null  float32
 9   prediction_level  97268 non-null  object 
dtypes: float32(3), int64(1), object(6)
memory usage: 6.3+ MB


In [27]:
#create new column
wpdx_prediction_df['status_prediction']=wpdx_prediction_df['status_id']-wpdx_prediction_df['prediction']

#check
wpdx_prediction_df['status_prediction'].head()

0    0.224805
1    0.255882
2    0.468150
3    0.358532
4    0.160506
Name: status_prediction, dtype: float64